# Convolutional Neural Network

**畳み込みニューラルネットワーク（Convolutional Neural Network, CNN）** は，主に画像データに対して高い性能を発揮する順伝播型ニューラルネットワークであり，畳み込み層とPooling層を複数積み重ねることで局所的な特徴から大域的な特徴を抽出し，その特徴から最終的に線形層から分類や回帰を行う．

このノートブックでは，シンプルなCNNを構築し，MLPと同様にダミーデータを用いて順伝播を計算する．

PyTorchでは，CNNに関しても，`nn.Module` を使ってニューラルネットワークの構造をクラスとして次のように定義することが一般的である．今回は，RGB画像を想定した3チャネルの画像を入力として受け取り，2回の畳み込みとPoolingを行い，特徴マップをベクトル化して，線形層に入力する構造とする．また畳み込みの後にBatch Normalizationを適用する．

これは次のように実装できる．

In [1]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = self.gap(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

これを `in_channels=3`，`num_classes=10` としてインスタンス化する．

In [2]:
in_channels = 3
num_classes = 10
model = CNN(in_channels=in_channels, num_classes=num_classes)

モデルの構造を確認する．

In [ ]:
print(model)

今回は活性化関数を `nn.functional.relu` で作成した．これは，`nn.ReLU` とは異なり，インスタンス化が不要な関数であり，関数として直接呼び出すことができる．

`nn.ReLU` は明示的にモデル内部に活性化関数を定義することができ，`nn.ReLU(inplace=True)` とするとメモリ効率を向上することができるが，基本的にはどちらの実装でも良い．

続いて，MLPと同様に正しく順伝播できるかどうかをダミーデータを与えてチェックしよう．今回は，$32 \times 32$ サイズの入力を考える．

In [ ]:
batch_size = 3
height, width = 32, 32
input_size = (batch_size, in_channels, height, width)
dummy_input = torch.randn(*input_size)

print('dummy_input.shape:', dummy_input.shape)
print('dummy_input = ')
print(dummy_input)

バッチサイズも含めて4回のテンソルが作成できた．順伝播を行い，(バッチサイズ, クラス数)の出力が得られることを確認する．

In [ ]:
output = model(dummy_input)

print('output.shape: ', output.shape)
print('output = ')
print(output)

このときエラーが発生する場合は `forward` 関数内の処理（特に，ベクトル化の処理）か畳み込みの入出力チャネル数を間違えていることが多い．

層へのアクセスやパラメータ数の取得方法も確認しておく．基本的にはMLPの場合と同じである．

In [ ]:
total_params = 0
for name, param in model.named_parameters():
    print(f'{name} shape: {param.shape}')
    total_params += param.numel()

print(f'Total number of trainable parameters: {total_params}')